In [379]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import glob
from nltk.stem.porter import PorterStemmer
from itertools import chain #untuk flaten array 2d

In [380]:
def readFile(directory):
    listFile = glob.glob(directory+"/*.txt")
    return listFile

## normalisasi

In [381]:
def normalisasi(body):

    #hapus tag
    hapusTag = r'<DATE>|</DATE>|<TITLE>|</TITLE>|<BODY>|</BODY>'
    body = re.sub(hapusTag, ' ', body)

    #hapus \n
    hapusEnter = r'\n'
    body = re.sub(hapusEnter, ' ', body)

    # hapus htmlEntities 
    htmlEntities = r"&lt;"
    body = re.sub(htmlEntities, '', body)

    kutip2 = r'[\"\(\)\<\>]'
    body = re.sub(kutip2, '', body)

    # hapus titik dan koma pada akhir string
    titikDiAkhirString = r"([a-zA-Z]+)([^\w\s])"
    temp = re.findall(titikDiAkhirString, body)
    x = True
    while x :
        temp = re.findall(titikDiAkhirString, body)
        if len(temp) == 0 or len(temp) == 1:
            x = False
        for i in temp:
            temp2 = i[0] + i[1]
            body = re.sub(temp2, i[0], body)

    # hapus titik dan koma pada akhir angka    
    titikSetelahAngka = r'(\d+)(^\w)\s'    
    temp = re.findall(titikSetelahAngka,body)
    while x :
        temp = re.findall(titikSetelahAngka, body)
        if len(temp) == 0 or len(temp) == 1:
            x = False
        for i in temp:
            temp2 = i[0] + i[1]
            body = re.sub(temp2, i[0], body)

    kutips = r"'s"
    body = re.sub(kutips, '', body)
    
    spasiLebihDariSatu = r'\s+'
    body = re.sub(spasiLebihDariSatu, ' ', body)
    return body

In [382]:
def readText(file):
    words = []
    f = open(file, 'r') #open file
    text = f.read()    
    f.close()

    return text

In [383]:
def removeEnter(listOfWords): #['<BODY>\n']
    for i in range(len(listOfWords)):
        listOfWords[i] = listOfWords[i].replace('\n','')
        
    return listOfWords

In [384]:
def removeNull(listOfWords): #['','','eat','food','','']
    listOfWords = list(filter(None, listOfWords))
    
    return listOfWords

In [385]:
def caseFolding(listOfWords):
    for i in range(len(listOfWords)):
        listOfWords[i] = listOfWords[i].casefold()
        
    return listOfWords

<h3>Load Stop Word from NLTK</h3>

In [386]:
stop_words = stopwords.words('english')

In [387]:
def removeStopWord(listOfWords):
    for i in listOfWords:
        if i in stop_words:
            listOfWords.remove(i)
        
    return listOfWords

<h3>Load Lemmatizer from NLTK</h3>

In [388]:
lemmatizer = WordNetLemmatizer()

In [389]:
def lemmatization(listOfWords):
    for i in range(len(listOfWords)):
        listOfWords[i] = lemmatizer.lemmatize(listOfWords[i])
        
    return listOfWords

<h3>Load Stemming from NLTK</h3>

In [390]:
def stemming(listOfWords):
    porter = PorterStemmer()
    stemmed = [porter.stem(word) for word in listOfWords]
    
    return stemmed

# Token

In [391]:
def make_token(txt):
    list_hasil = txt.split(" ")            
    return list_hasil

# Term

In [392]:
def make_term(file_name_token,file_name_term):
    list_term = []
    f = open(file_name_token, 'r')
    text = f.read()
    f.close()
    
    list_token = text.split("\n")
    for i in list_token:
        if i not in list_term:
            list_term.append(i)
            
    f1 = open(file_name_term,'a+')
    text = concate_term_newline(list_term)
    print(text)    
    f1.write(text)
    f1.close()


In [393]:
def concate_term_newline(list_in):
    hasil = ''
    for i in list_in:
        hasil = hasil + "\n " + i
        
    return hasil

# tulis data

In [394]:
def writeFile(list_in,file_name):
    list_flatten = list(chain.from_iterable(list_in))
        
    str_of_list = concate_term_newline(list_flatten)
    
    saveFileToken = open(file_name, 'a+' )
    saveFileToken.write(str_of_list)
    saveFileToken.close()
#     print(list_hasil)
    

<h2>Add Doc Number</h2>

In [395]:
def addDocNumber(_listIn, _docNum):
#     _docNum[14:18]
    for i in range(len(_listIn)):
        _listIn[i] = _listIn[i] + '_' + _docNum[14:18]
        
    return _listIn

<h1>Main</h1>

In [396]:
listOfFiles = readFile('DataRouter')
listOfWordsOfFile = []
counter = 1
blok = 10
list_token_sebelum = []
list_token_sesudah = []
list_term_sebelum = []
list_term_sesudah = []
blok_file_name = "term_blok"
blok_name = []

for i in listOfFiles: #iterasi tiap file
    txt = readText(i)
    list_token_sebelum.append(make_token(txt))
    txt = normalisasi(txt)
    listOfWords = make_token(txt)
    listOfWords = removeNull(listOfWords)
    listOfWords = caseFolding(listOfWords)
    listOfWords = removeStopWord(listOfWords)
    listOfWords = lemmatization(listOfWords)
#     listOfWords = stemming(listOfWords)
    
    # Diganti KEVIN
    #listOfWords = addDocNumber(listOfWords,i)
    
    list_token_sesudah.append(listOfWords)
    counter = counter + 1
#     print(counter)

    if (counter%blok) == 0:        
        writeFile(list_token_sebelum,"token_sebelum_pemprosesan.txt")
        writeFile(list_token_sesudah,"token_sesudah_pemprosesan.txt")
        
        #melakukan inisialisasi list dari, agar tidak memakan terlalu banyak memory
        listofWords = []
        list_token_sebelum = []
        list_token_sesudah = []
        list_term_sebelum = []
        list_term_sesudah = []
        
make_term("token_sebelum_pemprosesan.txt","term_sebelum_pemprosesan.txt")
make_term("token_sesudah_pemprosesan.txt","term_sesudah_pemprosesan.txt")
    

essed
  pumping
  260,000
  cartel
  exported
  173,500
  buildan
  pipelinecosting
  colombian
  saidestimated
  squeezed
  138
  jaime
  lusinchi
  saidzuela
  hector
  hurtado
  interior
  coordinate
  09:33:35.86
  m3
  moey
  roe
  oe
  o7
  oficial
  09:35:55.99
  greece
  piling
  paralyzing
  unprecedented
  handful
  olympic
  before
  trafficin
  out
  athens
  canceled
  saidheavy
  snowfall
  cold
  09:39:30.30
  malaysia
  rubber
  inra
  geneva
  hammer
  agreementlast
  lim
  keng
  yaik
  prepare
  strengthen
  orderly
  buffer
  capable
  absorbing
  09:39:45.70
  pharmacia
  internationalise
  suisse
  stanley
  svenska
  handelsbanken
  treasurer
  bertil
  tiusanen
  gaining
  internationally
  stockholm
  counter
  09:40:54.41
  mcdowell
  interpharm
  mcdowellenterprises
  19.9
  contigencies
  mcdowellcould
  fouryear
  mcdowells
  mcdowellboard
  09:41:09.67
  mortage
  swapped
  mortages
  guarantor
  convert
  investmentquality
  mortgagebacked
  collateral
  